In [1112]:
import numpy as np
import random
import copy

In [ ]:
# Initialize the game with n decks, the more decks, the harder it is to win
def initialize(n):
    global pre_deck,deck
    pre_deck = copy.deepcopy(list([11,2,3,4,5,6,7,8,9,10,10,10,10])*n*4)
    deck = copy.deepcopy(pre_deck)
    random.shuffle(deck)

# Deal the initial hands. Order doesn't matter here.
def deal():
    global dealer,player,deck
    dealer = []
    player = [[]]
    dealer.append(deck.pop(0))
    dealer.append(deck.pop(0))
    player[0].append(deck.pop(0))
    player[0].append(deck.pop(0))

# Check if a hand is blackjack (only works for initial 2 card hand)
def blackjack(hand):
    if 10 in hand and 11 in hand:
        return True
    else:
        return False

# Play as the dealer, happens after the player is finished
def action_dealer():
    global dealer,deck
    while sum(dealer)<17:
        dealer.append(deck.pop(0))
        if sum(dealer)>21 and (11 in dealer):
            dealer[dealer.index(11)]=1
        
# This game allows for one time splits. The "perfect strategy" logic is pre-programmed.
def split():
    global player,dealer,deck
    if player[0][0]==player[0][1]:
        if player[0][0]==11 or player[0][0]== 8:
            player.append([player[0][0],deck.pop(0)])
            player[0] = [player[0][0],deck.pop(0)]
        elif player[0][0]==9 and (not(dealer[0]==7 or dealer[0]==10 or dealer[0]==11)):
            player.append([player[0][0],deck.pop(0)])
            player[0] = [player[0][0],deck.pop(0)]
        elif player[0][0]==7 and (not(dealer[0]>=8 or dealer[0]==11)):
            player.append([player[0][0],deck.pop(0)])
            player[0] = [player[0][0],deck.pop(0)]
        elif player[0][0]==6 and (dealer[0]==3 or dealer[0]==4 or dealer[0]==5 or dealer[0]==6):
            player.append([player[0][0],deck.pop(0)])
            player[0] = [player[0][0],deck.pop(0)]
        elif player[0][0]==3 and (dealer[0]==7 or dealer[0]==4 or dealer[0]==5 or dealer[0]==6):
            player.append([player[0][0],deck.pop(0)])
            player[0] = [player[0][0],deck.pop(0)]
        else:
            return False
        return True
    else:
        return False

# Preprogrammed logic for when to double down
def double():
    global player,dealer,deck
    if len(player)==1 and len(player[0])==2:
        if 11 in player[0]:
            if (sum(player[0])==18 or sum(player[0])==17) and (3<=dealer[0]<=6):
                return True
            if (sum(player[0])==16 or sum(player[0])==15) and (4<=dealer[0]<=6):
                return True
            if (sum(player[0])==14 or sum(player[0])==13) and (5<=dealer[0]<=6):
                return True
        else:
            if sum(player[0])==11 and dealer[0]!=11:
                return True
            if sum(player[0])==10 and (2<=dealer[0]<=9):
                return True
            if sum(player[0])==9 and (3<=dealer[0]<=6):
                return True
    return(False)
            
# Preprogrammed logic for when to hit (applies to either hand in a split)
def hit(i):
    global player,dealer,deck
    if 11 in player[i]:
        if sum(player[i])>=19:
            return False
        if sum(player[i])==18 and (2<=dealer[0]<=8):
            return False
        else:
            return True
    else:
        if sum(player[i])>=17:
            return False
        if sum(player[i])>=13 and dealer[0]<=6:
            return False
        if sum(player[i])==12 and (4<=dealer[0]<=6):
            return False
        else:
            return True

# Score the player's hand against the dealer
def score(hand,bet):
    global dealer
    action_dealer()
    if sum(hand)>21:
        return -1*bet
    if sum(dealer)>21 or sum(hand)>sum(dealer):
        return bet
    if sum(hand)==sum(dealer):
        return 0
    else:
        return -1*bet

#1 = player, -1 = dealer, 0 = split
def play(bet):
    global dealer,player,deck
    if blackjack(dealer):
        if blackjack(player[0]):
            return 0
        else:
            return -1*bet
    elif blackjack(player[0]):
        # Blackjack pays 3:2 here
        return bet*1.5
    if (len(player)==1) and (len(player[0])==2):
        split()
    if double():
        player[0].append(deck.pop(0))
        if sum(player[0])>21:
            player[0][player[0].index(11)]=1
        if sum(player[0])>21:
            player[0][player[0].index(11)]=1
        return 2*score(player[0],bet)

    while True:
        any_hits = False
        for i in range(len(player)):
            if hit(i):
                any_hits=True
                player[i].append(deck.pop(0))
                if sum(player[i])>21 and 11 in player[i]:
                    player[i][player[i].index(11)]=1
                if sum(player[i])>21 and 11 in player[i]:
                    player[i][player[i].index(11)]=1
        if any_hits==False:
            temp_score = 0
            for i in range(len(player)):
                temp_score+=score(player[i],bet)
            return temp_score

In [ ]:
# Example game with 6 decks and 50% deck penetration
running_total = 0
base_bet = 1
initialize(6)
count=0
base_count = len([c for c in pre_deck if c <= 6]) - len([c for c in pre_deck if c >= 10])
for i in range(1000000):

    # Reshuffle after 50% deck penetration
    if len(deck)<(.5*len(pre_deck)):
        initialize(6)

    # This count is based on the cards left in the deck, but it works out the same because there are the same # of value 6 or less and value 10 or more in the deck
    # The -1* instead of just switching the order reflects the backwards approach of counting remaining instead of counting "seen"
    count = -1*round(((len([c for c in deck if c <= 6]) - len([c for c in deck if c >= 10]))/(((len(deck)))/52)))

    # This would wait for more deck penetration before betting big
    #if round(len(pre_deck)/52)==6:
     #   count = count-20
    # I think you'd need the rest (5,4,3) as well for this next elif to make sense?
    #elif round(len(pre_deck)/52)==2:
     #   count = count-4

    if count<=1:
        bet = base_bet*1
    elif count==2:
        bet = base_bet*2
    elif count==3:
        bet = base_bet*5
    elif count==4:
        bet = base_bet*5
    elif count>=5:
        bet = base_bet*10
        
    deal()
    
    # Keep the print statements commented out for large # of hands
    #print(player,dealer)
    running_total+=play(bet)
    #print(player,dealer)
print(running_total)
print(count,bet)

3892.0
-1 1


In [ ]:
#Expected return is $3/1000 $1 hands
# 0.1% Risk over 1000 $1 hands is $300 (simulating 1000 hands 1000 times yeilds max negative of $300)
#Estimated bankroll should be 500x base bet